# Colab disconnect probings

# <img src="http://reconstrue.com/assets/images/reconstrue_logo_brandmark.svg" width="42px" align="top" /> **Reconstrue**

This code is licensed by Reconstrue under the Apache 2.0 License.

## Introduction

This notebook's scope is Colab and its disconnect policies. The system will be probed in several ways to try to determine what the cut-off policies are.

For more context see the project's repo on GitHub, [@reconstrue/neuro_colab](https://github.com/reconstrue/neuro_colab).

## User perspective of Colab system

Rumors that should be tested:
- [ ] Idle disconnect (30m, 90m, what?)
- [x] JavaScript console can keep system from disconnecting
- [ ] Max 12 no matter what
- [ ] Max 2 notebooks running at once


Supposedly these Colab kernel (runtimes, VMs, whatever) are thrown away after a max of 12 hours. In the results of `uptime` is the time-of-day (formatted `HH:MM:SS`) followed by ` up: ` followed by how long the VM has been up (formatted as `XX min`, or after an hour as `H:MM`).

Note, the 12 hour lifetime cap is the max but less can happen it seems. Seemingly, [this issue is not well defined by google](https://stackoverflow.com/questions/55050988/can-i-run-a-google-colab-free-edition-script-and-then-shutdown-my-computer).

Furthermore, VM max lifetime is different than the constant runtime disconnects. These seem to happen on the order of 60 to 90 minutes. Supposedly keeping a code cell running can keep the connection from disconnect timeouts. Note though that editing text cells does not reset the timeout clock.

In [0]:
# How long has this VM been up? 
!uptime

## The JavaScript console technique

This idea seeded from Shivam Rawat's 2019-07-27 posting, [How to prevent Google Colab from disconnecting?](https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0). This code was tested on 2019-11-11 and got past 2 hours, after which the test was stopped.

Features:
- Pro
  - Short and sweet
- Con
  - Requires dropping into browser's JavaScript console
  - Dependent on DOM having a `colab-toolbar-button#connect`

Folks using this neuro_colab "platform" will mostly be writing code in Python. So, perhaps they are not familiar with web coding and the JavaScript console. 

If Google changes the DOM then this may eventually not work if they sufficiently change `colab-toolbar-button#connect`.

```
// TODO: want a way to detect that have been disconnected and console.log() that event's timestamp

var intervalInMilliseconds = 10000; // 1min = 60000; 5min = 300000 
var startTime = performance.now();

function ClickConnect(){
  var aMinuteInMilliseconds = 1000 * 60;
  var minutesConnected = (performance.now() - startTime) / aMinuteInMilliseconds;
  console.log("Connected duration in minutes: " + minutesConnected); 
  document.querySelector("colab-toolbar-button#connect").click(); 
}

setInterval(ClickConnect, intervalInMilliseconds);
```  



## A widget which does the same

Would be nice to have a widget that does the same. Maybe even a pip install-able one. Basically, don't want the easily triggered Pythonistas to have to deal directly with JavaScript. More seriously, just don't want to involve the dev console.

Should just be a bit of IPython display().


## Long running code

- [ ] Does simply running keep the connection alive?
- [ ] What if the progress bar is updated?

## Research bibliography

- [Does Google-Colab continue running the script when “Runtime disconnected”?](https://stackoverflow.com/questions/55874473/does-google-colab-continue-running-the-script-when-runtime-disconnected)
- [Can i run a google colab (free edition) script and then shutdown my computer?](https://stackoverflow.com/questions/55050988/can-i-run-a-google-colab-free-edition-script-and-then-shutdown-my-computer)
- [What's the hardware spec for Google Colaboratory?](https://stackoverflow.com/questions/47805170/whats-the-hardware-spec-for-google-colaboratory/47805171#47805171)